In [1]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from util.math import *
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [2]:
uso = yf.download("USO", start="2006-05-24", end="2012-04-09", progress=False)[
    "Adj Close"
].to_numpy()
gld = yf.download("GLD", start="2006-05-24", end="2012-04-09", progress=False)[
    "Adj Close"
].to_numpy()

In [3]:
lookback = 20
hedge_ratio = np.empty(len(uso))
hedge_ratio[:lookback] = np.nan

In [4]:
def get_hedge_ratio(x, y):
    _x = np.vstack((x, np.ones(len(x)))).T
    regression_result = np.linalg.lstsq(_x, y, rcond=None)
    return regression_result[0][0]


for t in range(lookback, hedge_ratio.shape[0]):
    x = gld[t - lookback : t]
    y = uso[t - lookback : t]
    hedge_ratio[t] = get_hedge_ratio(x, y)


In [5]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.arange(len(hedge_ratio)), y=hedge_ratio, mode="lines", name="lines")
)
fig.update_layout(title=dict(text="Hedge Ratio"))
fig.show("notebook")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
price_mat = np.column_stack((gld, uso))
weights = np.column_stack((-hedge_ratio, np.ones(len(hedge_ratio))))
portfolio = np.sum(price_mat * weights, axis=1)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.arange(len(portfolio)), y=portfolio, mode="lines", name="lines")
)
fig.update_layout(title=dict(text="Constructed Portfolio Market Value"))
fig.show()

In [ ]:
portfolio = portfolio[lookback:]
hedge_ratio = hedge_ratio[lookback:]
price_mat = price_mat[lookback:,]

In [ ]:
rolling_mean = calculate_rolling_mean(portfolio, 20)
rolling_std = calculate_rolling_std(portfolio, 20)

In [ ]:
n_units = -(portfolio - rolling_mean) / calculate_std(portfolio)

In [ ]:
positions = np.tile(n_units[:, np.newaxis], (1, price_mat.shape[1])) * np.column_stack(
    (-hedge_ratio, np.ones(len(hedge_ratio)))
)

In [ ]:
pnl = np.sum(shift(positions, 1) * calculate_daily_return(price_mat), axis=1)

In [ ]:
ret = pnl / np.sum(np.abs(shift(positions, 1)), axis=1)
ret[np.isnan(ret)] = 0

C:\Users\nileb\AppData\Local\Temp\ipykernel_27832\1041607093.py:1: RuntimeWarning:

invalid value encountered in divide



In [ ]:
apr = np.prod(1 + ret) ** (252 / len(ret)) - 1
sharpe = np.sqrt(252) * np.mean(ret) / np.std(ret)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(len(ret)), y=np.cumprod(1 + ret) - 1, mode="lines", name="lines"
    )
)
fig.update_layout(title=dict(text="Cumulative return"))
fig.show()

In [ ]:
print(f"APR={apr} Sharpe={sharpe}")

APR=0.11169180189316563 Sharpe=0.6428554726754407
